In [1]:
import pandas as pd
import numpy as np
import json
import os
import glob
from tqdm import tqdm

In [2]:
pd.set_option('display.max_rows', None)

In [3]:
path = '/home/manoelflorencio/cta_for_jd/testbedXS'
os.chdir(path)
print(os.getcwd())

/home/manoelflorencio/cta_for_jd/testbedXS


In [4]:
def generate_key_for_evaluation(df):
    left_table_name = []
    left_column_name = []
    right_table_name = []
    right_column_name = []
    df_key = pd.DataFrame((), columns=df.columns)
    
    for i in range(df.shape[0]):
        rtn = df.iloc[i, 2]
        ltn = df.iloc[i, 0]
        right_table_name.append(rtn if rtn > ltn else ltn)
        left_table_name.append(rtn if rtn < ltn else ltn)
    
        rcn = df.iloc[i, 3]
        lcn = df.iloc[i, 1]
        right_column_name.append(rcn if rtn > ltn else lcn)
        left_column_name.append(rcn if rtn < ltn else lcn)
    
    d = {'LEFT_TABLE':left_table_name,
         'LEFT_COLUMN':left_column_name,
         'RIGHT_TABLE':right_table_name,
         'RIGHT_COLUMN':right_column_name}
    df_key = pd.DataFrame(d)
    df_key = df_key.drop_duplicates()
    df_key = df_key.reset_index(drop=True)
    
    key = []
    for i in range(df_key.shape[0]):
        key.append("#".join(df_key.iloc[i,:]))
    df_key['KEY'] = key

    return df_key

In [5]:
def fix_table_names(table_name):
    return table_name + '.csv'

In [6]:
cta_matches = pd.read_csv('Description_test/CTA_from_descriptions/junio_JD_matches_with_table_descriptions.csv')
cta_matches = cta_matches[cta_matches['JOINABLE_TD'] == 'Yes'][['LEFT_TABLE','LEFT_COLUMN','RIGHT_TABLE','RIGHT_COLUMN']]
cta_matches.head()

,LEFT_TABLE,LEFT_COLUMN,RIGHT_TABLE,RIGHT_COLUMN
0,community-centres,ADDRESS,community-gardens-and-food-trees,MERGED_ADDRESS
1,community-centres,Geo Local Area,community-gardens-and-food-trees,Geo Local Area
2,community-centres,ADDRESS,cultural-spaces,ADDRESS
3,community-centres,Geo Local Area,cultural-spaces,LOCAL_AREA
6,community-centres,Geo Local Area,libraries,Geo Local Area


In [7]:
cta_matches['LEFT_TABLE'] = fix_table_names(cta_matches['LEFT_TABLE'])
cta_matches['RIGHT_TABLE'] = fix_table_names(cta_matches['RIGHT_TABLE'])

In [8]:
cta_matches = generate_key_for_evaluation(cta_matches)
cta_matches.head()

,LEFT_TABLE,LEFT_COLUMN,RIGHT_TABLE,RIGHT_COLUMN,KEY
0,community-centres.csv,ADDRESS,community-gardens-and-food-trees.csv,MERGED_ADDRESS,community-centres.csv#ADDRESS#community-garden...
1,community-centres.csv,Geo Local Area,community-gardens-and-food-trees.csv,Geo Local Area,community-centres.csv#Geo Local Area#community...
2,community-centres.csv,ADDRESS,cultural-spaces.csv,ADDRESS,community-centres.csv#ADDRESS#cultural-spaces....
3,community-centres.csv,Geo Local Area,cultural-spaces.csv,LOCAL_AREA,community-centres.csv#Geo Local Area#cultural-...
4,community-centres.csv,Geo Local Area,libraries.csv,Geo Local Area,community-centres.csv#Geo Local Area#libraries...


In [9]:
cta_matches.shape

(93, 5)

In [10]:
groundTruth = pd.read_csv('joinable_columns_gt3_quality.csv')
groundTruth = groundTruth[['ds_name', 'att_name', 'ds_name_2', 'att_name_2']]
groundTruth = generate_key_for_evaluation(groundTruth)
groundTruth.head()

,LEFT_TABLE,LEFT_COLUMN,RIGHT_TABLE,RIGHT_COLUMN,KEY
0,road-ahead-current-road-closures.csv,COMP_DATE,road-ahead-upcoming-projects.csv,COMP_DATE,road-ahead-current-road-closures.csv#COMP_DATE...
1,eo_pr.csv,NTEE_CD,eo_xx.csv,NTEE_CD,eo_pr.csv#NTEE_CD#eo_xx.csv#NTEE_CD
2,eo4.csv,NAME,eo_pr.csv,NAME,eo4.csv#NAME#eo_pr.csv#NAME
3,eo4.csv,ICO,eo_pr.csv,ICO,eo4.csv#ICO#eo_pr.csv#ICO
4,eo4.csv,STREET,eo_pr.csv,STREET,eo4.csv#STREET#eo_pr.csv#STREET


In [11]:
groundTruth.shape

(53, 5)

In [12]:
tp = 0
fp = 0
fn = 0

for i in range(len(cta_matches)):
    if(cta_matches.iloc[i,4] in groundTruth.iloc[:,4].values):
        tp += 1
    else:
        fp += 1
    
for i in range(len(groundTruth)):
    if(groundTruth.iloc[i,4] not in cta_matches.iloc[:,4].values):
        fn += 1
    
precision = (tp)/(tp+fp)
recall = (tp)/(tp+fn)
f1_score = 2 * (precision*recall)/(precision+recall)

print(precision, recall, f1_score)

0.27956989247311825 0.49056603773584906 0.35616438356164376


In [13]:
tp, fp, fn

(26, 67, 27)

# Falso Positivo

In [16]:
fp_rows = []
for i in range(len(cta_matches)):
    if(cta_matches.iloc[i,4] not in groundTruth.iloc[:,4].values):
        fp_rows.append(cta_matches.iloc[i,4])

fp_df = pd.DataFrame(list(map(lambda x: x.split('#'), fp_rows)), columns=['LEFT_TABLE','LEFT_COLUMN','RIGHT_TABLE','RIGHT_COLUMN'])
fp_df.to_csv('Description_test/CTA_from_descriptions/false_positives.csv', index=False)

# Falso Negativo

In [15]:
for i in range(len(groundTruth)):
    if(groundTruth.iloc[i,4] not in cta_matches.iloc[:,4].values):
        print(groundTruth.iloc[i,4])

eo4.csv#ICO#eo_pr.csv#ICO
eo4.csv#CITY#eo_pr.csv#CITY
eo4.csv#SORT_NAME#eo_pr.csv#SORT_NAME
cultural-spaces.csv#LOCAL_AREA#street-intersections.csv#Geo Local Area
cultural-spaces.csv#LOCAL_AREA#schools.csv#Geo Local Area
community-gardens-and-food-trees.csv#Geo Local Area#cultural-spaces.csv#LOCAL_AREA
rental-standards-current-issues.csv#Geo Local Area#street-intersections.csv#Geo Local Area
public-art.csv#Neighbourhood#street-intersections.csv#Geo Local Area
public-art.csv#GeoLocalArea#street-intersections.csv#Geo Local Area
community-gardens-and-food-trees.csv#Geo Local Area#street-intersections.csv#Geo Local Area
public-art.csv#Neighbourhood#rental-standards-current-issues.csv#Geo Local Area
public-art.csv#GeoLocalArea#rental-standards-current-issues.csv#Geo Local Area
rental-standards-current-issues.csv#Geo Local Area#schools.csv#Geo Local Area
community-gardens-and-food-trees.csv#Geo Local Area#rental-standards-current-issues.csv#Geo Local Area
public-art.csv#Neighbourhood#schools